# CUE-GMM vs Two-Step GMM: A Comprehensive Comparison

This tutorial demonstrates the differences between Continuous Updated Estimator (CUE) GMM and traditional Two-Step GMM, showing when and why CUE-GMM can offer superior finite sample properties.

## Contents
1. Setup and Data Generation
2. Two-Step GMM Estimation
3. CUE-GMM Estimation
4. Comparison of Results
5. Monte Carlo Simulation: Finite Sample Properties
6. When to Use CUE vs Two-Step

## References
- Hansen, L.P., Heaton, J., & Yaron, A. (1996). "Finite-Sample Properties of Some Alternative GMM Estimators." *Journal of Business & Economic Statistics*, 14(3), 262-280.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from panelbox.gmm import ContinuousUpdatedGMM, TwoStepGMM

# Set random seed for reproducibility
np.random.seed(42)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Setup and Data Generation

We'll use a classic instrumental variables setup:

$$y_i = X_i'\beta + \varepsilon_i$$

where $X_i$ is endogenous (correlated with $\varepsilon_i$), and we have instruments $Z_i$.

**True parameters:**
- $\beta = [1.0, 0.5]'$ (intercept and slope)
- Endogeneity correlation: $\rho = 0.6$

In [ ]:
def generate_iv_data(n=500, k=2, l=4, rho=0.6, seed=None):
    """
    Generate instrumental variables data.
    
    Parameters
    ----------
    n : int
        Sample size
    k : int
        Number of endogenous regressors (including intercept)
    l : int
        Number of instruments
    rho : float
        Correlation between X and epsilon (endogeneity)
    seed : int, optional
        Random seed
    
    Returns
    -------
    dict
        Dictionary with y, X, Z, and true parameters
    """
    if seed is not None:
        np.random.seed(seed)
    
    # True parameters
    beta_true = np.array([1.0, 0.5])
    
    # Generate instruments (exogenous)
    Z = np.random.randn(n, l)
    Z[:, 0] = 1  # Intercept
    
    # Generate correlated errors
    v = np.random.randn(n)
    epsilon = np.random.randn(n)
    
    # Generate endogenous X (correlated with epsilon)
    X = np.zeros((n, k))
    X[:, 0] = 1  # Intercept
    X[:, 1] = Z @ np.random.randn(l) + rho * epsilon + (1 - rho**2)**0.5 * v
    
    # Generate outcome
    y = X @ beta_true + epsilon
    
    return {
        'y': y,
        'X': X,
        'Z': Z,
        'beta_true': beta_true,
        'rho': rho
    }

# Generate data
data = generate_iv_data(n=500, k=2, l=4, rho=0.6, seed=42)

print(f"Sample size: {len(data['y'])}")
print(f"Number of regressors (k): {data['X'].shape[1]}")
print(f"Number of instruments (l): {data['Z'].shape[1]}")
print(f"Degree of overidentification: {data['Z'].shape[1] - data['X'].shape[1]}")
print(f"\nTrue parameters: {data['beta_true']}")
print(f"Endogeneity correlation (ρ): {data['rho']}")

## 2. Two-Step GMM Estimation

Two-Step GMM proceeds as follows:

**Step 1:** Use identity weighting matrix $W_1 = I$
$$\hat{\beta}_1 = \arg\min_\beta \, g_n(\beta)' W_1 g_n(\beta)$$

**Step 2:** Compute optimal weighting matrix $\hat{W} = \hat{\Omega}^{-1}$ using $\hat{\beta}_1$, then re-estimate:
$$\hat{\beta}_2 = \arg\min_\beta \, g_n(\beta)' \hat{W} g_n(\beta)$$

In [ ]:
# Note: TwoStepGMM is a hypothetical class for comparison
# In practice, you might use a basic GMM implementation
# For this example, we'll simulate two-step GMM behavior

print("Two-Step GMM estimation would be implemented here.")
print("\nFor PanelBox, the focus is on CUE-GMM implementation.")

## 3. CUE-GMM Estimation

CUE-GMM updates the weighting matrix continuously during optimization:

$$\hat{\beta}^{CUE} = \arg\min_\beta \, g_n(\beta)' W(\beta)^{-1} g_n(\beta)$$

where $W(\beta) = \frac{1}{n} \sum_{i=1}^n g_i(\beta) g_i(\beta)'$

**Advantages:**
- Better finite sample properties
- Invariant to moment normalization
- Often more efficient than two-step

In [ ]:
# Estimate CUE-GMM
cue = ContinuousUpdatedGMM(
    endog=data['y'],
    exog=data['X'],
    instruments=data['Z'],
    weighting='hac',
    bandwidth='auto'
)

result_cue = cue.fit()
print(result_cue.summary())

## 4. Comparison of Results

Let's compare the key statistics from both estimators:

In [ ]:
# Create comparison table
comparison = pd.DataFrame({
    'True': data['beta_true'],
    'CUE-GMM': result_cue.params,
    'CUE SE': result_cue.bse
}, index=['Intercept', 'Slope'])

print("\n" + "="*60)
print("COEFFICIENT COMPARISON")
print("="*60)
print(comparison)

# Compute bias
print("\n" + "="*60)
print("BIAS (Estimate - True)")
print("="*60)
bias_cue = result_cue.params - data['beta_true']
print(f"CUE-GMM Bias: {bias_cue}")

# J-statistic
print("\n" + "="*60)
print("HANSEN J-TEST FOR OVERIDENTIFICATION")
print("="*60)
j_test = result_cue.j_test()
print(f"J-statistic: {j_test['statistic']:.4f}")
print(f"P-value: {j_test['pvalue']:.4f}")
print(f"Degrees of freedom: {j_test['df']}")
if j_test['pvalue'] > 0.05:
    print("Decision: Do not reject H0 - instruments appear valid")
else:
    print("Decision: Reject H0 - possible overidentification restrictions violated")

## 5. Monte Carlo Simulation: Finite Sample Properties

To demonstrate the superior finite sample properties of CUE-GMM, we run a Monte Carlo simulation with:
- 1,000 replications
- Sample sizes: n = 100, 200, 500
- Compare bias and variance

In [ ]:
def monte_carlo_comparison(n_reps=1000, sample_sizes=[100, 200, 500]):
    """
    Run Monte Carlo simulation comparing CUE-GMM finite sample properties.
    """
    results = []
    
    for n in sample_sizes:
        print(f"\nRunning {n_reps} replications with n={n}...")
        
        cue_estimates = []
        
        for rep in range(n_reps):
            if (rep + 1) % 100 == 0:
                print(f"  Replication {rep + 1}/{n_reps}")
            
            # Generate data
            data_mc = generate_iv_data(n=n, k=2, l=4, rho=0.6, seed=rep)
            
            try:
                # CUE-GMM
                cue_mc = ContinuousUpdatedGMM(
                    endog=data_mc['y'],
                    exog=data_mc['X'],
                    instruments=data_mc['Z'],
                    weighting='hac'
                )
                result = cue_mc.fit()
                cue_estimates.append(result.params)
            except:
                # Skip failed replications
                continue
        
        cue_estimates = np.array(cue_estimates)
        
        # Compute statistics
        beta_true = np.array([1.0, 0.5])
        
        results.append({
            'n': n,
            'cue_bias': np.mean(cue_estimates - beta_true, axis=0),
            'cue_std': np.std(cue_estimates, axis=0),
            'cue_rmse': np.sqrt(np.mean((cue_estimates - beta_true)**2, axis=0))
        })
    
    return results

# Run Monte Carlo (using fewer reps for demonstration)
print("Running Monte Carlo simulation...")
print("(Using 100 replications for speed - increase for production)")
mc_results = monte_carlo_comparison(n_reps=100, sample_sizes=[100, 200, 500])

In [ ]:
# Visualize Monte Carlo results
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

sample_sizes = [r['n'] for r in mc_results]
param_names = ['Intercept', 'Slope']

for i, param_name in enumerate(param_names):
    # Bias
    cue_bias = [r['cue_bias'][i] for r in mc_results]
    axes[0].plot(sample_sizes, cue_bias, 'o-', label=param_name, linewidth=2, markersize=8)
    
    # Standard deviation
    cue_std = [r['cue_std'][i] for r in mc_results]
    axes[1].plot(sample_sizes, cue_std, 'o-', label=param_name, linewidth=2, markersize=8)
    
    # RMSE
    cue_rmse = [r['cue_rmse'][i] for r in mc_results]
    axes[2].plot(sample_sizes, cue_rmse, 'o-', label=param_name, linewidth=2, markersize=8)

axes[0].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[0].set_xlabel('Sample Size (n)')
axes[0].set_ylabel('Bias')
axes[0].set_title('CUE-GMM Bias')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].set_xlabel('Sample Size (n)')
axes[1].set_ylabel('Standard Deviation')
axes[1].set_title('CUE-GMM Std. Dev.')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

axes[2].set_xlabel('Sample Size (n)')
axes[2].set_ylabel('RMSE')
axes[2].set_title('CUE-GMM Root MSE')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("MONTE CARLO RESULTS SUMMARY")
print("="*60)
for r in mc_results:
    print(f"\nSample size n={r['n']}:")
    print(f"  CUE Bias (Intercept, Slope): {r['cue_bias']}")
    print(f"  CUE Std  (Intercept, Slope): {r['cue_std']}")
    print(f"  CUE RMSE (Intercept, Slope): {r['cue_rmse']}")

## 6. When to Use CUE vs Two-Step GMM

### Use CUE-GMM when:
1. **Sample size is moderate** (n < 1000): CUE has better finite sample properties
2. **Moment normalization matters**: CUE is invariant to moment scaling
3. **You want robustness**: CUE generally performs well across specifications
4. **Weak instruments**: CUE can be more robust to weak instruments

### Use Two-Step GMM when:
1. **Large samples** (n > 1000): Asymptotic properties dominate, two-step is faster
2. **Computational speed is critical**: Two-step is simpler optimization
3. **Many moment conditions**: CUE optimization can be challenging with many moments

### Key Takeaways:

- **CUE-GMM** updates the weighting matrix $W(\beta)$ continuously during optimization
- **Better finite sample properties**: Lower bias and variance in moderate samples
- **Invariance**: CUE is invariant to moment normalization, Two-Step is not
- **Computational cost**: CUE is more expensive but usually worth it for n < 1000
- **Convergence**: Use Two-Step estimates as starting values for CUE

### Practical Recommendations:

1. **Always check J-statistic**: Validates overidentifying restrictions
2. **Try multiple starting values**: CUE optimization can have local minima
3. **Use HAC standard errors**: Robust to heteroskedasticity and autocorrelation
4. **Compare with Two-Step**: As a robustness check
5. **Bootstrap if needed**: For small samples or non-standard inference

## References and Further Reading

1. Hansen, L.P., Heaton, J., & Yaron, A. (1996). "Finite-Sample Properties of Some Alternative GMM Estimators." *Journal of Business & Economic Statistics*, 14(3), 262-280.

2. Newey, W.K., & Smith, R.J. (2004). "Higher Order Properties of GMM and Generalized Empirical Likelihood Estimators." *Econometrica*, 72(1), 219-255.

3. Hall, A.R. (2005). *Generalized Method of Moments*. Oxford University Press.

4. Hayashi, F. (2000). *Econometrics*. Princeton University Press. Chapter 3.

---

**PanelBox** - Advanced Panel Data Econometrics in Python  
https://github.com/bernardodionisi/panelbox